In [1]:
import pandas as pd
from swmm_api.input_file import read_inp_file
from swmm_api.input_file.section_labels import *
from swmm_api.input_file.sections import *
from swmm_api.input_file.helpers import *
from swmm_api.input_file.sections.others import TimeseriesData
from swmm_api.input_file.section_labels import TIMESERIES,LID_USAGE
import os
os.environ['CONDA_DLL_SEARCH_MODIFICATION_ENABLE']="1"
import numpy as np
import shutil
import yaml
from pyswmm import Simulation,Links,Nodes,RainGages,SystemStats
import matplotlib.pyplot as plt
import step_sim
import GI_setting as gs

e:\anaconda3\envs\torch-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = yaml.load(open(os.path.dirname(os.getcwd())+"\\states_yaml\\chaohu.yaml"), yaml.FullLoader)
rain = np.load(os.path.dirname(os.getcwd())+'\\rainfall\\training_raindata.npy').tolist()
inp = read_inp_file(os.path.dirname(os.getcwd())+'\\SWMM\\chaohu_RTC.inp')


# 设置GI方案，考虑30%、50%、70%比例

In [4]:
for rainid in range(10):
    inp1 = inp.copy()
    inp1[TIMESERIES]['rainfall']=TimeseriesData('rainfall',rain[rainid])
    GIarea = gs.get_GIarea(inp,0.3)
    inp1 = gs.set_GIusage(inp1,GIarea,10)
    inp1.write_file('./Results/chaohu_rain_'+str(rainid)+'.inp')
    #step_sim.simfile('./S12/Results/RSH/',rainid,config)
    sim=Simulation('./Results/chaohu_rain_'+str(rainid)+'.inp')
    #sim.execute()
    sim.start()
    time = sim._model.swmm_stride(300)
    sim._model.swmm_end()
    sim._model.swmm_close()